A stock-movement prediction model for the company  *UpUpUp Inc*. 

Authors:

*   Eva Ósk Gunnarsdóttir (evag18@ru.is)
*   Justyna Micota (justyna22@ru.is)

In [200]:
# Set flexable variables to play with
BATCH_SIZE = 10
NN_WIDTH = 32
LR = 0.001
N_EPOCHS = 10

# Fetch data and provided functions

Here we fetch the given functions and data (stock pricees, market analysis, market segments and info)

In [201]:
# Import necessary libraries
import torch
import torch.nn as nn
import pandas as pd
from google.colab import drive
from os.path import join
# Mounting Google Drive

ROOT = '/content/drive'
drive.mount(ROOT, force_remount=True)

# Copy data to local runtime
# NOTE: PROJ might need to be customized

PROJ = 'MyDrive/DEEP-Assignment1'  
PROJ_PATH = join(ROOT, PROJ)
!rsync -aP "{PROJ_PATH}"/* ./

import simulator
import stock_preprocessing as spp


Mounted at /content/drive
sending incremental file list


# Preprocess/ prepare data


Here we join the information of the given data (stock pricees, market analysis, market segments and info) to a single table.

Stock_preprocessing module contains process_training_data method, that standardizes data, and generates train and test Dataloaders for training purposes.

In [202]:
stock_prices = []
market_analysis = []
market_segments = []
info = []

simulator.read_stock_prices(stock_prices)
simulator.read_market_analysis(market_analysis)
simulator.read_market_segments(market_segments)
simulator.read_info(info)

train_dl, test_dl = spp.process_training_data(market_segments, market_analysis, 
                                              info, stock_prices, BATCH_SIZE)

# Build the Neural Network Model

In [203]:
# Create Neural Network Model (ANN)
class StocksModel(nn.Module):
  
  def __init__(self):
    super().__init__()
    # TODO: play with Depth
    self.fc1= nn.Linear(len(X[0])*len(X[0][0]), NN_WIDTH)
    self.av1= nn.ReLU()
    self.fc2= nn.Linear(NN_WIDTH, NN_WIDTH)
    self.av2= nn.ReLU()
    self.fc3= nn.Linear(NN_WIDTH, NN_WIDTH)
    self.av3= nn.ReLU()
    self.fc4= nn.Linear(NN_WIDTH, NN_WIDTH)
    self.av4= nn.ReLU()
    self.fc5= nn.Linear(NN_WIDTH, NN_WIDTH)
    self.av5= nn.ReLU()
    self.fc6= nn.Linear(NN_WIDTH, 8) 
    self.avout= nn.LogSoftmax(dim=-1)

    return None

  def forward(self, x):
    x = self.fc1(x)
    x = self.av1(x)
    x = self.fc2(x)
    x = self.av2(x)
    x = self.fc3(x)
    x = self.av3(x)
    x = self.fc4(x)
    x = self.av4(x)
    x = self.fc5(x)
    x = self.av5(x)
    x = self.fc6(x)
    x = self.avout(x)      # Alternate way: nn.functional.log_softmax(x)
    return x

stock_market_predictor_0000 = StocksModel()
print(stock_market_predictor_0000)

StocksModel(
  (fc1): Linear(in_features=33, out_features=32, bias=True)
  (av1): ReLU()
  (fc2): Linear(in_features=32, out_features=32, bias=True)
  (av2): ReLU()
  (fc3): Linear(in_features=32, out_features=32, bias=True)
  (av3): ReLU()
  (fc4): Linear(in_features=32, out_features=32, bias=True)
  (av4): ReLU()
  (fc5): Linear(in_features=32, out_features=32, bias=True)
  (av5): ReLU()
  (fc6): Linear(in_features=32, out_features=8, bias=True)
  (avout): LogSoftmax(dim=-1)
)


# Train Model

In [204]:
optimizer = torch.optim.Adam(stock_market_predictor_0000.parameters(), lr=LR)

for epochs in range(N_EPOCHS):
  for data in train_dl:
    X, y = data
    optimizer.zero_grad() # clear gradient information.
    output = stock_market_predictor_0000(X.view(-1, len(X[0])*len(X[0][0])))
    loss = nn.functional.nll_loss(output, y)
    loss.backward() # do pack-propagation step
    optimizer.step() # update weights

# Evaluate

In [205]:
total = 0
correct = 0
with torch.no_grad():   # No need for keepnig track of necessary changes to the gradient.
  for data in test_dl:
    X, y = data
    output = stock_market_predictor_0000(X.view(-1, len(X[0])*len(X[0][0])))
    for idx, val in enumerate(output):
      if torch.argmax(val) == y[idx]:
        correct += 1
      total += 1
  print('Accuracy:', round(correct/total, 3))

Accuracy: 0.55


# Check Predictor compatibility

In [206]:
import predictor

predictor_object = predictor.Predictor('stocks_predictor', 
                                       stock_market_predictor_0000)
simulator.simulate(2017, 1, predictor_object)

# predictor_object.predict(market_segments, market_analysis, info, [1,1,1,1])


[(0, 'IT'), (1, 'BIO'), (2, 'IT')] [('IT', 2017, 0, -1), ('BIO', 2017, 0, 1)] [(0, 2017, 4, 0, 0, 1, 10, 5.1, 6912.0, -0.9, 0), (1, 2017, 4, 0, 1, 1, 5, -4.8, 2128.0, -0.4, 0), (2, 2017, 4, 0, 0, 0, 0, 0.1, 7482.0, 0.9, 0)] [102.2, 99.5, 98.3]
Predictions (year, day): 2017 4 (False, False, False) Target: [False, True, False]
[(0, 'IT'), (1, 'BIO'), (2, 'IT')] [('IT', 2017, 0, -1), ('BIO', 2017, 0, 1)] [(0, 2017, 5, 0, 0, 1, 10, 6.6, 8928.0, 0.3, 0), (1, 2017, 5, 0, 0, 1, 5, -4.9, 8232.0, 0.4, 0), (2, 2017, 5, 0, 0, 0, 0, 5.9, 9977.0, 0.7, 0)] [102.2, 100.7, 97.8]
Predictions (year, day): 2017 5 (False, False, False) Target: [False, True, False]
[(0, 'IT'), (1, 'BIO'), (2, 'IT')] [('IT', 2017, 0, -1), ('BIO', 2017, 0, 1)] [(0, 2017, 6, 0, 0, 1, 10, 7.8, 6924.0, 0.0, 0), (1, 2017, 6, 0, 0, 0, 5, 3.4, 5388.0, 0.8, 0), (2, 2017, 6, 0, 0, 0, 0, -9.1, 5421.0, -0.9, 0)] [102.2, 100.7, 97.8]
Predictions (year, day): 2017 6 (False, False, False) Target: [False, False, False]
[(0, 'IT'), (1, 'BI